# Reto 2
Construye un modelo básico de preguntas y respuestas.

## Fuente de datos.


## Resumen de la implementación

In [3]:
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW
from torch.utils.data import DataLoader, Dataset
import torch

# Definir la clase del conjunto de datos de preguntas y respuestas
class QADataset(Dataset):
    def __init__(self, preguntas, respuestas, tokenizer):
        self.preguntas = preguntas
        self.respuestas = respuestas
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.preguntas)

    def __getitem__(self, idx):
        encoding = self.tokenizer.encode_plus(
            self.preguntas[idx],
            self.respuestas[idx],
            add_special_tokens=True,
            truncation=True,
            max_length=512,
            padding='max_length',
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'start_positions': torch.tensor([encoding['input_ids'].squeeze().tolist().index(1)]),
            'end_positions': torch.tensor([encoding['input_ids'].squeeze().tolist().index(1) + len(self.respuestas[idx].split()) - 1])
        }

# Datos de ejemplo
preguntas = [
    "¿Qué es la inteligencia artificial?",
    "¿En qué se utiliza la inteligencia artificial?",
    "¿Cuáles son algunas aplicaciones de la inteligencia artificial?"
]
respuestas = [
    "La inteligencia artificial (IA) es un campo de estudio multidisciplinario que busca desarrollar algoritmos y sistemas capaces de imitar el razonamiento y la toma de decisiones humanas.",
    "La inteligencia artificial se utiliza en una amplia variedad de aplicaciones, como reconocimiento de voz, chatbots, conducción autónoma y mucho más.",
    "Algunas aplicaciones de la inteligencia artificial incluyen reconocimiento facial, asistentes virtuales, sistemas de recomendación, entre otros."
]

# Inicializar el tokenizador y el modelo
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
modelo = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

# Crear el conjunto de datos y el DataLoader
dataset = QADataset(preguntas, respuestas, tokenizer)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Configurar el optimizador
optimizador = AdamW(modelo.parameters(), lr=2e-5)

# Entrenamiento del modelo
for epoch in range(3):
    for batch in dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        start_positions = batch['start_positions']
        end_positions = batch['end_positions']

        modelo.zero_grad()
        outputs = modelo(input_ids=input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        loss.backward()
        optimizador.step()

# Guardar el modelo entrenado
modelo.save_pretrained('ruta/al/modelo/entrenado')
tokenizer.save_pretrained('ruta/al/tokenizador/entrenado')


/Users/aceron/opt/miniconda3/envs/courses/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-18 19:58:32.047164: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', '

ValueError: 1 is not in list